In [2]:
import torch
import torch.nn as nn
from rfft_lib import rfft_audio
import onnx
import math

In [3]:
audio_path = "/home/satya-tt0492/Documents/TT/ex-7/audio/sax.wav"

In [4]:
audio = rfft_audio.load_audio(audio_path)
print(audio)

tensor([ 0.0053,  0.0062,  0.0075,  ..., -0.0001, -0.0001, -0.0002])


In [5]:
audio.shape

torch.Size([57618])

In [6]:
temp_input = torch.rand(16)
temp_input

tensor([0.7049, 0.1233, 0.7220, 0.5830, 0.1711, 0.6032, 0.9263, 0.7348, 0.7738,
        0.8988, 0.4135, 0.4347, 0.2317, 0.4669, 0.3767, 0.1061])

In [7]:
rfft_audio.apply_rfft(temp_input)

tensor([8.2707, 1.7070, 0.9202, 0.6153, 0.6040, 1.1199, 1.2618, 1.1066, 0.3693])

In [8]:
def fft(samples):
    N = len(samples)

    if N == 1:
        return samples
    
    M = N // 2

    Xeven = [0. for i in range(M)]
    Xodd = [0. for i in range(M)]

    for i in range(M):
        Xeven[i] = samples[2 * i]
        Xodd[i] = samples[2 * i + 1]
    
    Feven = [0. for i in range(M)]
    Feven = fft(Xeven)

    Fodd = [0. for i in range(M)]
    Fodd = fft(Xodd)

    freqbin = [0. for i in range(N)]

    for k in range(N // 2):
        cmplx = complex(1,-2 * math.pi * k / N) * Fodd[k]

        freqbin[k] = Feven[k] + cmplx
        freqbin[k + N // 2] = Feven[k] - cmplx

    return abs(torch.tensor(freqbin))

In [9]:
ans = fft(temp_input)

In [10]:
abs(torch.fft.fft(temp_input))

tensor([8.2707, 1.7070, 0.9202, 0.6153, 0.6040, 1.1199, 1.2618, 1.1066, 0.3693,
        1.1066, 1.2618, 1.1199, 0.6040, 0.6153, 0.9202, 1.7070])

In [11]:
abs(torch.tensor(ans))

/tmp/ipykernel_235996/2932582670.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  abs(torch.tensor(ans))


tensor([8.2707, 4.1898, 1.6523, 6.6004, 0.8716, 3.1958, 1.4449, 8.7046, 0.3693,
        1.1924, 0.9558, 3.7811, 0.4891, 2.0737, 0.9641, 7.2168])

In [12]:
temp_input.shape[0]

16

In [13]:
def fft(samples):
    samples = torch.tensor(samples)

    N = samples.shape[0]

    if N == 1:
        return samples
    
    M = N // 2

    Xeven = torch.zeros(M)
    Xodd = torch.zeros(M)

    for i in range(M):
        Xeven[i] = samples[2 * i]
        Xodd[i] = samples[2 * i + 1]
    
    Feven = torch.zeros(M)
    Feven = fft(Xeven)

    Fodd = torch.zeros(M)
    Fodd = fft(Xodd)

    freqbin = torch.zeros(N)

    for k in range(N // 2):
        cmplx = complex(1,-2 * math.pi * k / N) * Fodd[k]

        freqbin[k] = Feven[k] + cmplx
        freqbin[k + N // 2] = Feven[k] - cmplx

    return abs(freqbin)

In [14]:
temp_input = torch.rand(1024*1024*256)

In [15]:
x = abs(torch.fft.fft(temp_input))

In [16]:
y = fft(temp_input)

/tmp/ipykernel_235996/1756566828.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  samples = torch.tensor(samples)
/tmp/ipykernel_235996/1756566828.py:29: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /opt/conda/conda-bld/pytorch_1724788959732/work/aten/src/ATen/native/Copy.cpp:305.)
  freqbin[k] = Feven[k] + cmplx


KeyboardInterrupt: 

In [29]:
torch.mean(abs(x-y))

tensor(7.1705)